## How to run the experiments

Run the code blocs bellow in sequence. You can read the descriptions to understand it.


The dependencies can be found in https://github.com/eduardogc8/simple-qc

Before starting to run the experiments, change the variable ``path_wordembedding``, in the code block below, for the correct directory path. Make sure that the word embedding inside follow the template `wiki.multi.*.vec`.

In [1]:
import nltk
import numpy as np
import pandas as pd
from keras.preprocessing.sequence import pad_sequences
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import normalize

from benchmarking_methods import run_benchmark
from building_classifiers import lstm_default, svm_linear, random_forest, cnn
from download_word_embeddings import muse_embeddings_path, download_if_not_existing
from loading_data import load_embedding, load_uiuc

path_wordembedding = '/home/eduardo/word_embedding/'
download_if_not_existing()
from benchmarking_methods import run_benchmark_cv
from feature_creation import create_feature
from loading_data import load_disequa

Using TensorFlow backend.


### Extract features

The function *create_features* transform the questions in numerical vector to a classifier model.<br>It returns the output in the df_2 dataframe that is a parameter (*df_2.feature_type*, according to the *feature_type*).<br><br>
**feature_type:** type of feature. (bow, tfidf, embedding, embedding_sum, vocab_index, pos_index, pos_hotencode, ner_index, ner_hotencode)<br> 
**df:** the dataframe used to fit the transformers models (df.questions).<br>
**df_2:** dataframe wich the data will be transformed (df_2.questions).<br>
**embedding:** embedding model for word embedding features type.<br>
**max_features:** used in bag-of-words and TFIDF.


### Create classifier models

The models are created through functions that return them. These functions will be used to create a new model in each experiment. Therefore, an instance of a model is created by the benchmark function and not explicitly in a code block.


### UTILS

In [5]:
import warnings
warnings.filterwarnings("ignore")



#### Load UIUC dataset

#### Load DISEQuA dataset

## Benchmark UIUC - Normal

**Normal:** it uses the default fixed split of UIUC between train dataset (at last 5500 instances) and test dataset (500 instances). Therefore, it does not use cross-validation.

When the *run_benchmark* function is executed, it will save each result in the *save* path.

**model:** a dictionary with the classifier name and the function to create and return the model (not an instance of the model). <br> Example: *model = {'name': 'SVM', 'model': svm_linear}*<br>
**X:** all the training set.<br>
**y:** all the labels of the training set.<br>
**x_test:** test set.<br>
**y_test:** labels of the test set.<br>
**sizes_train:** sizes of training set. For each size, an experiment is executed.<br>
**runs:** number of time that each experiment is executed (used in models which has parameters with random values, like weights in an ANN).<br>
**save:** csv path where the results will be saved.<br>
**metric_average:** used in f1, recall and precision metrics<br>
**onehot:** one-hot model to transform labels.<br>
**out_dim:** the total of classes for ANN models.<br>
**epochs:** epochs for ANN models.<br>
**batch_size:** batch_size for ANN models.<br>
**vocabulary_size:** vocabulary size (used in CNN model).



## Benchmark UIUC and DISEQuA - Cross-validation

**Cross-validation:** instead of uses default fixed splits, it uses the all the dataset with cross-validation with 10 folds.

When the *run_benchmark* function is executed, it will save each result in the *save* path.

**model:** a dictionary with the classifier name and the function to create and return the model (not an instance of the model). <br> Example: *model = {'name': 'SVM', 'model': svm_linear}*<br>
**X:** Input features.<br>
**y:** Input labels.<br>
**sizes_train:** sizes of training set. For each size, an experiment is executed.<br>
**folds:** Amount of folds for cross-validations.<br>
**save:** csv path where the results will be saved.<br>
**metric_average:** used in f1, recall and precision metrics<br>
**onehot:** one-hot model to transform labels.<br>
**epochs:** epochs for ANN models.<br>
**batch_size:** batch_size for ANN models.<br>
**vocabulary_size:** vocabulary size (used in CNN model).



## Run UIUC Benchmark - Normal

Different classifier models are tested with different dependency levels of external linguistic resources (Low, Medium and High)

#### SVM + TF-IDF

In [ ]:
for language in ['en', 'es', 'pt']:
    print('\n\nLanguage: ', language)
    dataset_train, dataset_test = load_uiuc(language)
    create_feature('tfidf', dataset_train, dataset_train, max_features=2000)
    create_feature('tfidf', dataset_train, dataset_test, max_features=2000)
    
    model = {'name': 'svm', 'model': svm_linear}
    
    tfidf_train = np.array([list(r) for r in dataset_train['tfidf'].values])
    tfidf_test = np.array([list(r) for r in dataset_test['tfidf'].values])
    tfidf_train = normalize(tfidf_train, norm='max')
    tfidf_test = normalize(tfidf_test, norm='max')
    
    X_train = np.array([list(x) for x in dataset_train['tfidf'].values])
    X_test = np.array([list(x) for x in dataset_test['tfidf'].values])
    y_train = dataset_train['class'].values
    y_test = dataset_test['class'].values
    
    run_benchmark(model, X_train, y_train, X_test, y_test, sizes_train=[1000, 2000, 3000, 4000, 5500],
                  save='results/UIUC_svm_tfidf_' + language + '.csv', runs=1)

#### SVM + TF-IDF + WB

In [ ]:
for language in ['en', 'es', 'pt']:
    print('\n\nLanguage: ', language)
    embedding = load_embedding(path_wordembedding + 'wiki.multi.' + language + '.vec')
    dataset_train, dataset_test = load_uiuc(language)
    create_feature('tfidf', dataset_train, dataset_train, max_features=2000)
    create_feature('tfidf', dataset_train, dataset_test, max_features=2000)
    create_feature('embedding_sum', None, dataset_train, embedding)
    create_feature('embedding_sum', None, dataset_test, embedding)
    
    model = {'name': 'svm', 'model': svm_linear}
    
    tfidf_train = np.array([list(r) for r in dataset_train['tfidf'].values])
    tfidf_test = np.array([list(r) for r in dataset_test['tfidf'].values])
    tfidf_train = normalize(tfidf_train, norm='max')
    tfidf_test = normalize(tfidf_test, norm='max')
    
    embedding_train = np.array([list(r) for r in dataset_train['embedding_sum'].values])
    embedding_test = np.array([list(r) for r in dataset_test['embedding_sum'].values])
    embedding_train = normalize(embedding_train, norm='max')
    embedding_test = normalize(embedding_test, norm='max')
    
    X_train = np.array([list(x) + list(xx) for x, xx in zip(tfidf_train, embedding_train)])
    X_test = np.array([list(x) + list(xx) for x, xx in zip(tfidf_test, embedding_test)])
    y_train = dataset_train['class'].values
    y_test = dataset_test['class'].values
    
    run_benchmark(model, X_train, y_train, X_test, y_test, sizes_train=[1000, 2000, 3000, 4000, 5500], 
                  runs=1, save='results/UIUC_svm_cortes_' + language + '.csv')

#### SVM + TF-IDF + WB + POS + NER

In [ ]:
for language in ['en', 'es', 'pt']:
    print('\n\nLanguage: ', language)
    embedding = load_embedding(path_wordembedding + 'wiki.multi.' + language + '.vec')
    dataset_train, dataset_test = load_uiuc(language)
    create_feature('tfidf', dataset_train, dataset_train, max_features=2000)
    create_feature('tfidf', dataset_train, dataset_test, max_features=2000)
    create_feature('embedding_sum', dataset_train, dataset_train, embedding)
    create_feature('embedding_sum', dataset_train, dataset_test, embedding)
    create_feature('pos_hotencode', dataset_train, dataset_train)
    create_feature('pos_hotencode', dataset_train, dataset_test)
    create_feature('ner_hotencode', dataset_train, dataset_train)
    create_feature('ner_hotencode', dataset_train, dataset_test)
    model = {'name': 'svm', 'model': svm_linear}
    
    tfidf_train = np.array([list(r) for r in dataset_train['tfidf'].values])
    tfidf_test = np.array([list(r) for r in dataset_test['tfidf'].values])
    tfidf_train = normalize(tfidf_train, norm='max')
    tfidf_test = normalize(tfidf_test, norm='max')
    
    embedding_train = np.array([list(r) for r in dataset_train['embedding_sum'].values])
    embedding_test = np.array([list(r) for r in dataset_test['embedding_sum'].values])
    embedding_train = normalize(embedding_train, norm='max')
    embedding_test = normalize(embedding_test, norm='max')
    
    pos_train = np.array([list(r) for r in dataset_train['pos_hotencode'].values])
    pos_test = np.array([list(r) for r in dataset_test['pos_hotencode'].values])
    
    ner_train = np.array([list(r) for r in dataset_train['ner_hotencode'].values])
    ner_test = np.array([list(r) for r in dataset_test['ner_hotencode'].values])
    
    X_train = np.array([list(x) + list(xx) + list(xxx) + list(xxxx) for x, xx, xxx, xxxx in zip(tfidf_train, embedding_train, pos_train, ner_train)])
    X_test = np.array([list(x) + list(xx) + list(xxx) + list(xxxx) for x, xx, xxx, xxxx in zip(tfidf_test, embedding_test, pos_test, ner_test)])
    
    y_train = dataset_train['class'].values
    y_test = dataset_test['class'].values
    
    classes = list(dataset_train['class'].unique())
    y_train_ = [classes.index(c) for c in y_train]
    
    run_benchmark(model, X_train, y_train, X_test, y_test, sizes_train=[1000, 2000, 3000, 4000, 5500],
                  runs=1, save='results/UIUC_svm_high_' + language + '.csv')

#### BERT + CNN

In [3]:
from typing import List
from flair_cnn_doc_embedding import DocumentCNNEmbeddings
from torch.utils.data import Dataset
import torch
from flair.data import Sentence, Corpus
from flair.embeddings import DocumentRNNEmbeddings, BertEmbeddings
from flair.models import TextClassifier
from flair.trainers import ModelTrainer
import time
import datetime
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, matthews_corrcoef, confusion_matrix


def build_flair_sentences(text_label_tuples):
    sentences = [Sentence(text, labels=[label], use_tokenizer=True) for text,label in text_label_tuples]
    return [s for s in sentences if len(s.tokens) > 0]

def get_labels(sentences:List[Sentence]):
    return [[l.value for l in s.labels] for s in sentences]


def calc_metrics_with_sklearn(clf:TextClassifier,sentences:List[Sentence],train_size=0,
                              run=0,train_time=0,metric_average='macro',
                              classes=['ABBR', 'DESC', 'ENTY', 'HUM', 'LOC', 'NUM']):
    targets = get_labels(sentences)
    start_time = time.time()
    clf.predict(sentences)
    test_time = time.time() - start_time
    prediction = get_labels(sentences)
    p = prediction
    t = targets
    data = {'datetime': datetime.datetime.now(),
            'model': 'cnn_bert',
            'accuracy': accuracy_score(prediction, targets),
            'precision': precision_score(prediction, targets, average=metric_average),
            'recall': recall_score(prediction, targets, average=metric_average),
            'f1': f1_score(prediction, targets, average=metric_average),
            'mcc': matthews_corrcoef(prediction, targets),
            'confusion': confusion_matrix(prediction, targets, labels=classes),
            'run': run,
            'train_size': size_train,
            'execution_time': train_time,
            'test_time': test_time}
          
    #report = metrics.classification_report(y_true=targets, y_pred=prediction, digits=3, output_dict=True)
    return data


for language in ['es']: # ,  'pt', 'en', 
    results = pd.DataFrame()
    
    save = 'results/UIUC_cnn_bert_'+language+'_1000.csv'
    for size_train in [5500]:  # , 2000, 3000, 4000, 5500
        for run in range(3):
            dataset_train, dataset_test = load_uiuc(language)
            if size_train < 5500:
                dataset_train = dataset_train[:size_train]

            sentences_train:Dataset = build_flair_sentences([(text, label) for text, label in zip(dataset_train['question'], dataset_train['class'])])
            sentences_dev:Dataset = sentences_train
            sentences_test:Dataset = build_flair_sentences([(text, label) for text, label in zip(dataset_test['question'], dataset_test['class'])])

            corpus:Corpus = Corpus(sentences_train, sentences_dev, sentences_test)
            label_dict = corpus.make_label_dictionary()
            word_embeddings = [
                # WordEmbeddings('glove'),
                BertEmbeddings('bert-base-multilingual-cased', layers='-1')
            ]
            document_embeddings = DocumentCNNEmbeddings(word_embeddings,
                                                        dropout=0.0,
                                                        hidden_size=64,
                                                        )

            clf = TextClassifier(document_embeddings, label_dictionary=label_dict, multi_label=False)
            trainer = ModelTrainer(clf, corpus,torch.optim.RMSprop)
            base_path = 'flair_resources/qc_en_uiuc'
            start_time = time.time()
            trainer.train(base_path,
                          learning_rate=0.001,
                          mini_batch_size=32,
                          anneal_factor=0.5,
                          patience=2,
                          max_epochs=5)
            train_time = time.time() - start_time
            data = calc_metrics_with_sklearn(clf, sentences_test, train_size=size_train, train_time=train_time, run=run)
            results = results.append([data])
            results.to_csv(save)

2019-09-05 14:41:19,847 {'ENTY', 'NUM', 'HUM', 'LOC', 'DESC', 'ABBR'}
2019-09-05 14:41:19,849 The pre-trained model you are loading is a cased model but you have not set `do_lower_case` to False. We are setting `do_lower_case=False` for you but you may want to check this behavior.
2019-09-05 14:41:28,877 ----------------------------------------------------------------------------------------------------
2019-09-05 14:41:28,878 Evaluation method: MICRO_F1_SCORE
2019-09-05 14:41:29,124 ----------------------------------------------------------------------------------------------------
2019-09-05 14:41:30,507 epoch 1 - iter 0/169 - loss 2.06307292
2019-09-05 14:41:45,111 epoch 1 - iter 16/169 - loss 5.45063688
2019-09-05 14:41:59,127 epoch 1 - iter 32/169 - loss 3.54964152
2019-09-05 14:42:13,487 epoch 1 - iter 48/169 - loss 2.80683617
2019-09-05 14:42:28,718 epoch 1 - iter 64/169 - loss 2.37665758
2019-09-05 14:42:43,123 epoch 1 - iter 80/169 - loss 2.10667268
2019-09-05 14:42:58,509 epo

/home/eduardo/.virtualenvs/qa/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/eduardo/.virtualenvs/qa/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


2019-09-05 15:12:13,033 {'ENTY', 'NUM', 'HUM', 'LOC', 'DESC', 'ABBR'}
2019-09-05 15:12:13,035 The pre-trained model you are loading is a cased model but you have not set `do_lower_case` to False. We are setting `do_lower_case=False` for you but you may want to check this behavior.
2019-09-05 15:12:22,279 ----------------------------------------------------------------------------------------------------
2019-09-05 15:12:22,280 Evaluation method: MICRO_F1_SCORE
2019-09-05 15:12:22,646 ----------------------------------------------------------------------------------------------------
2019-09-05 15:12:24,285 epoch 1 - iter 0/169 - loss 1.82131362
2019-09-05 15:12:39,698 epoch 1 - iter 16/169 - loss 4.42343454
2019-09-05 15:12:56,109 epoch 1 - iter 32/169 - loss 3.10937216
2019-09-05 15:13:11,623 epoch 1 - iter 48/169 - loss 2.56547279
2019-09-05 15:13:27,484 epoch 1 - iter 64/169 - loss 2.22186843
2019-09-05 15:13:43,082 epoch 1 - iter 80/169 - loss 1.98553506
2019-09-05 15:13:58,464 epo

/home/eduardo/.virtualenvs/qa/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/eduardo/.virtualenvs/qa/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


2019-09-05 15:43:47,515 {'ENTY', 'NUM', 'HUM', 'LOC', 'DESC', 'ABBR'}
2019-09-05 15:43:47,519 The pre-trained model you are loading is a cased model but you have not set `do_lower_case` to False. We are setting `do_lower_case=False` for you but you may want to check this behavior.
2019-09-05 15:44:08,649 ----------------------------------------------------------------------------------------------------
2019-09-05 15:44:08,650 Evaluation method: MICRO_F1_SCORE
2019-09-05 15:44:09,073 ----------------------------------------------------------------------------------------------------
2019-09-05 15:44:10,811 epoch 1 - iter 0/169 - loss 2.38130069
2019-09-05 15:44:26,511 epoch 1 - iter 16/169 - loss 7.63982942
2019-09-05 15:44:40,799 epoch 1 - iter 32/169 - loss 4.96618783
2019-09-05 15:44:57,404 epoch 1 - iter 48/169 - loss 3.85119579
2019-09-05 15:45:13,428 epoch 1 - iter 64/169 - loss 3.16408118
2019-09-05 15:45:29,232 epoch 1 - iter 80/169 - loss 2.73323929
2019-09-05 15:45:44,920 epo

/home/eduardo/.virtualenvs/qa/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/eduardo/.virtualenvs/qa/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


## Run UIUC Benchmark - Cross-validation

Different classifier models are tested with different dependency levels of external linguistic resources (Low, Medium and High)

#### SVM + TF-IDF

In [ ]:
for language in ['en', 'es', 'pt']:
    print('\n\nLanguage: ', language)
    dataset_train, dataset_test = load_uiuc(language)
    dataset = pd.concat([dataset_train, dataset_test])
    create_feature('tfidf', dataset, dataset, max_features=2000)
    
    model = {'name': 'svm', 'model': svm_linear}
    
    tfidf = np.array([list(r) for r in dataset['tfidf'].values])
    tfidf = normalize(tfidf, norm='max')
    
    X = np.array([list(x) for x in dataset['tfidf'].values])
    y = dataset['class'].values
    
    
    # run_benchmark_cv(model, X, y, [50, 100] + list(range(500, 5501, 500)),
    run_benchmark_cv(model, X, y, [1000, 2000, 3000, 4000, 5500],
                     save='results/UIUC_cv_svm_tfidf_' + language + '.csv')

#### SVM + TF-IDF + WB

In [ ]:
for language in ['en', 'es', 'pt']:
    print('\n\nLanguage: ', language)
    embedding = load_embedding(path_wordembedding + 'wiki.multi.' + language + '.vec')
    dataset_train, dataset_test = load_uiuc(language)
    dataset = pd.concat([dataset_train, dataset_test])
    create_feature('tfidf', dataset, dataset, max_features=2000)
    create_feature('embedding_sum', None, dataset, embedding)
    
    model = {'name': 'svm', 'model': svm_linear}
    
    tfidf = np.array([list(r) for r in dataset['tfidf'].values])
    tfidf = normalize(tfidf, norm='max')
    
    embedding = np.array([list(r) for r in dataset['embedding_sum'].values])
    embedding = normalize(embedding, norm='max')
    
    X = np.array([list(x) + list(xx) for x, xx in zip(tfidf, embedding)])
    y = dataset['class'].values
    
    # run_benchmark_cv(model, X, y, [50, 100] + list(range(500, 5501, 500)),
    run_benchmark_cv(model, X, y, [1000, 2000, 3000, 4000, 5500],
                     save='results/UIUC_cv_svm_cortes_' + language + '.csv')

#### SVM + TF-IDF + WB + POS + NER

In [ ]:
for language in ['en', 'es', 'pt']:
    print('\n\nLanguage: ', language)
    embedding = load_embedding(path_wordembedding + 'wiki.multi.' + language + '.vec')
    dataset_train, dataset_test = load_uiuc(language)
    dataset = pd.concat([dataset_train, dataset_test])
    create_feature('tfidf', dataset, dataset, max_features=2000)
    create_feature('embedding_sum', dataset, dataset, embedding)
    create_feature('pos_hotencode', dataset, dataset)
    create_feature('ner_hotencode', dataset, dataset)
    model = {'name': 'svm', 'model': svm_linear}
    
    tfidf = np.array([list(r) for r in dataset['tfidf'].values])
    tfidf = normalize(tfidf, norm='max')
    
    embedding = np.array([list(r) for r in dataset['embedding_sum'].values])
    embedding = normalize(embedding, norm='max')
    
    pos = np.array([list(r) for r in dataset['pos_hotencode'].values])
    
    ner = np.array([list(r) for r in dataset['ner_hotencode'].values])
    
    X = np.array([list(x) + list(xx) + list(xxx) + list(xxxx) for x, xx, xxx, xxxx in zip(tfidf, embedding, pos, ner)])
    
    y = dataset['class'].values
    
    # run_benchmark_cv(model, X, y, [50, 100] + list(range(500, 5501, 500)),
    run_benchmark_cv(model, X, y, [1000, 2000, 3000, 4000, 5500],
                     save='results/UIUC_cv_svm_high_' + language + '.csv')

#### BERT + CNN - Cross validation

In [6]:
from typing import List
from flair_cnn_doc_embedding import DocumentCNNEmbeddings
from torch.utils.data import Dataset
import torch
from flair.data import Sentence, Corpus
from flair.embeddings import DocumentRNNEmbeddings, BertEmbeddings
from flair.models import TextClassifier
from flair.trainers import ModelTrainer
import time
import datetime
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, matthews_corrcoef, confusion_matrix


def build_flair_sentences(text_label_tuples):
    sentences = [Sentence(text, labels=[label], use_tokenizer=True) for text,label in text_label_tuples]
    return [s for s in sentences if len(s.tokens) > 0]

def get_labels(sentences:List[Sentence]):
    return [[l.value for l in s.labels] for s in sentences]

def calc_metrics_with_sklearn(clf:TextClassifier,sentences:List[Sentence],train_size=0,
                              fold=0,train_time=0,metric_average='macro',
                              classes=['ABBR', 'DESC', 'ENTY', 'HUM', 'LOC', 'NUM']):
    targets = get_labels(sentences)
    start_time = time.time()
    clf.predict(sentences)
    test_time = time.time() - start_time
    prediction = get_labels(sentences)
    p = prediction
    t = targets
    data = {'datetime': datetime.datetime.now(),
            'model': 'cnn_bert',
            'accuracy': accuracy_score(prediction, targets),
            'precision': precision_score(prediction, targets, average=metric_average),
            'recall': recall_score(prediction, targets, average=metric_average),
            'f1': f1_score(prediction, targets, average=metric_average),
            'mcc': matthews_corrcoef(prediction, targets),
            'confusion': confusion_matrix(prediction, targets, labels=classes),
            'fold': fold,
            'train_size': size_train,
            'execution_time': train_time,
            'test_time': test_time}

    #report = metrics.classification_report(y_true=targets, y_pred=prediction, digits=3, output_dict=True)
    return data




for language in ['es',]: # ,  'es', 'pt'
    print(f"########## {language} ##########")
    results = pd.DataFrame()
    dataset_train, dataset_test = load_uiuc(language)
    dataset = pd.concat([dataset_train, dataset_test])
    save = 'results/UIUC_cv_cnn_bert_'+language+'_1000_2000.csv'
    for size_train in [1000, 2000]: # 
        print(f"##### {size_train} #####")
        
        word_embeddings = [BertEmbeddings('bert-base-multilingual-cased', layers='-1')]
        document_embeddings = DocumentCNNEmbeddings(word_embeddings, dropout=0.0, hidden_size=64)
        
        size_test = len(dataset) - size_train
        rs = StratifiedShuffleSplit(n_splits=10, train_size=size_train, test_size=size_test, random_state=1)
        fold = 0
        for train_indexs, test_indexs in rs.split(dataset, dataset['class']):
            fold += 1
            print(f"## {fold} ##")
            df_train = dataset.iloc[train_indexs]
            df_test = dataset.iloc[test_indexs]
            
            x_train:Dataset = build_flair_sentences([(text, label) for text, label in zip(df_train['question'], df_train['class'])])
            x_dev:Dataset = x_train
            x_test:Dataset = build_flair_sentences([(text, label) for text, label in zip(df_test['question'], df_test['class'])])
            
            corpus = Corpus(x_train, x_dev, x_test)
            label_dict = corpus.make_label_dictionary()

            clf = TextClassifier(document_embeddings, label_dictionary=label_dict, multi_label=False)
            trainer = ModelTrainer(clf, corpus,torch.optim.RMSprop)
            base_path = 'flair_resources/qc_'+language+'_uiuc'
            start_time = time.time()
            trainer.train(base_path,
                          learning_rate=0.001,
                          mini_batch_size=32,
                          anneal_factor=0.5,
                          max_epochs=5,
                          patience=2,
                          )
            train_time = time.time() - start_time
            data = calc_metrics_with_sklearn(clf, x_test, train_size=size_train, train_time=train_time, fold=fold)
            results = results.append([data])
            results.to_csv(save)

########## es ##########
##### 1000 #####
2019-08-30 00:15:56,544 The pre-trained model you are loading is a cased model but you have not set `do_lower_case` to False. We are setting `do_lower_case=False` for you but you may want to check this behavior.
## 1 ##
2019-08-30 00:16:44,314 {'ENTY', 'NUM', 'ABBR', 'HUM', 'DESC', 'LOC'}
2019-08-30 00:16:44,526 ----------------------------------------------------------------------------------------------------
2019-08-30 00:16:44,527 Evaluation method: MICRO_F1_SCORE
2019-08-30 00:16:45,244 ----------------------------------------------------------------------------------------------------
2019-08-30 00:16:47,386 epoch 1 - iter 0/32 - loss 1.77986097
2019-08-30 00:16:50,576 epoch 1 - iter 3/32 - loss 8.04128671
2019-08-30 00:16:53,477 epoch 1 - iter 6/32 - loss 6.44657620
2019-08-30 00:16:56,890 epoch 1 - iter 9/32 - loss 5.41995763
2019-08-30 00:16:59,525 epoch 1 - iter 12/32 - loss 4.59803794
2019-08-30 00:17:02,758 epoch 1 - iter 15/32 - lo

2019-08-30 00:41:46,920 epoch 2 - iter 12/32 - loss 1.03961400
2019-08-30 00:41:49,447 epoch 2 - iter 15/32 - loss 1.00217039
2019-08-30 00:41:52,242 epoch 2 - iter 18/32 - loss 0.99930125
2019-08-30 00:41:55,333 epoch 2 - iter 21/32 - loss 1.03038987
2019-08-30 00:41:58,089 epoch 2 - iter 24/32 - loss 1.02364323
2019-08-30 00:42:01,085 epoch 2 - iter 27/32 - loss 0.98668079
2019-08-30 00:42:04,376 epoch 2 - iter 30/32 - loss 0.98622822
2019-08-30 00:42:06,090 ----------------------------------------------------------------------------------------------------
2019-08-30 00:42:06,092 EPOCH 2 done: loss 0.9702 - lr 0.0010 - bad epochs 0
2019-08-30 00:42:34,942 DEV : loss 0.6005484461784363 - score 0.776
2019-08-30 00:45:11,661 TEST : loss 0.9332205653190613 - score 0.672
2019-08-30 00:45:18,071 ----------------------------------------------------------------------------------------------------
2019-08-30 00:45:19,903 epoch 3 - iter 0/32 - loss 0.68040985
2019-08-30 00:45:22,861 epoch 3 -

2019-08-30 01:14:09,416 epoch 4 - iter 0/32 - loss 0.29436827
2019-08-30 01:14:12,317 epoch 4 - iter 3/32 - loss 0.36847810
2019-08-30 01:14:16,698 epoch 4 - iter 6/32 - loss 0.38930078
2019-08-30 01:14:19,076 epoch 4 - iter 9/32 - loss 0.39506946
2019-08-30 01:14:22,146 epoch 4 - iter 12/32 - loss 0.42571741
2019-08-30 01:14:24,620 epoch 4 - iter 15/32 - loss 0.43147716
2019-08-30 01:14:26,987 epoch 4 - iter 18/32 - loss 0.45169947
2019-08-30 01:14:29,940 epoch 4 - iter 21/32 - loss 0.46752783
2019-08-30 01:14:32,542 epoch 4 - iter 24/32 - loss 0.48703005
2019-08-30 01:14:35,146 epoch 4 - iter 27/32 - loss 0.48262915
2019-08-30 01:14:37,769 epoch 4 - iter 30/32 - loss 0.48836879
2019-08-30 01:14:51,805 ----------------------------------------------------------------------------------------------------
2019-08-30 01:14:51,992 EPOCH 4 done: loss 0.4909 - lr 0.0010 - bad epochs 0
2019-08-30 01:15:19,717 DEV : loss 0.3683728873729706 - score 0.866
2019-08-30 01:17:58,523 TEST : loss 0.722

2019-08-30 01:42:18,577 ----------------------------------------------------------------------------------------------------
## 5 ##
2019-08-30 01:44:56,002 {'ENTY', 'NUM', 'ABBR', 'HUM', 'DESC', 'LOC'}
2019-08-30 01:44:56,019 ----------------------------------------------------------------------------------------------------
2019-08-30 01:44:56,020 Evaluation method: MICRO_F1_SCORE
2019-08-30 01:44:56,674 ----------------------------------------------------------------------------------------------------
2019-08-30 01:44:59,045 epoch 1 - iter 0/32 - loss 8.46961308
2019-08-30 01:45:01,958 epoch 1 - iter 3/32 - loss 4.47838008
2019-08-30 01:45:05,070 epoch 1 - iter 6/32 - loss 3.46463159
2019-08-30 01:45:08,097 epoch 1 - iter 9/32 - loss 2.94123491
2019-08-30 01:45:10,924 epoch 1 - iter 12/32 - loss 2.61017027
2019-08-30 01:45:13,875 epoch 1 - iter 15/32 - loss 2.43157306
2019-08-30 01:45:17,212 epoch 1 - iter 18/32 - loss 2.27864429
2019-08-30 01:45:20,545 epoch 1 - iter 21/32 - loss 

2019-08-30 02:13:38,011 epoch 2 - iter 18/32 - loss 0.73612964
2019-08-30 02:13:40,875 epoch 2 - iter 21/32 - loss 0.74855922
2019-08-30 02:13:43,803 epoch 2 - iter 24/32 - loss 0.74200044
2019-08-30 02:13:46,585 epoch 2 - iter 27/32 - loss 0.74170662
2019-08-30 02:13:49,176 epoch 2 - iter 30/32 - loss 0.76656455
2019-08-30 02:13:50,114 ----------------------------------------------------------------------------------------------------
2019-08-30 02:13:50,115 EPOCH 2 done: loss 0.7595 - lr 0.0010 - bad epochs 0
2019-08-30 02:14:17,572 DEV : loss 0.642583966255188 - score 0.772
2019-08-30 02:16:55,673 TEST : loss 0.7775426506996155 - score 0.7306
2019-08-30 02:17:03,503 ----------------------------------------------------------------------------------------------------
2019-08-30 02:17:05,275 epoch 3 - iter 0/32 - loss 0.71338159
2019-08-30 02:17:08,624 epoch 3 - iter 3/32 - loss 0.63500477
2019-08-30 02:17:11,665 epoch 3 - iter 6/32 - loss 0.59191082
2019-08-30 02:17:14,069 epoch 3 - i

...

2019-08-30 06:59:13,901 epoch 4 - iter 12/63 - loss 0.41851586
2019-08-30 06:59:20,802 epoch 4 - iter 18/63 - loss 0.45900025
2019-08-30 06:59:26,353 epoch 4 - iter 24/63 - loss 0.43404794
2019-08-30 06:59:31,665 epoch 4 - iter 30/63 - loss 0.43845882
2019-08-30 06:59:37,607 epoch 4 - iter 36/63 - loss 0.43892490
2019-08-30 06:59:44,319 epoch 4 - iter 42/63 - loss 0.43396895
2019-08-30 07:01:28,265 epoch 4 - iter 48/63 - loss 0.43914601
2019-08-30 07:01:42,805 epoch 4 - iter 54/63 - loss 0.42909163
2019-08-30 07:02:40,623 epoch 4 - iter 60/63 - loss 0.44225843
2019-08-30 07:03:22,029 ----------------------------------------------------------------------------------------------------
2019-08-30 07:03:22,573 EPOCH 4 done: loss 0.4414 - lr 0.0010 - bad epochs 0
2019-08-30 07:04:34,845 DEV : loss 0.3516794741153717 - score 0.888
2019-08-30 07:07:01,847 TEST : loss 0.6665229201316833 - score 0.7995
2019-08-30 07:07:16,327 ---------------------------------------------------------------------

2019-08-30 07:45:45,857 ----------------------------------------------------------------------------------------------------
## 7 ##
2019-08-30 07:48:54,339 {'ENTY', 'NUM', 'ABBR', 'HUM', 'DESC', 'LOC'}
2019-08-30 07:48:54,388 ----------------------------------------------------------------------------------------------------
2019-08-30 07:48:54,390 Evaluation method: MICRO_F1_SCORE
2019-08-30 07:48:55,311 ----------------------------------------------------------------------------------------------------
2019-08-30 07:48:57,344 epoch 1 - iter 0/63 - loss 5.33133459
2019-08-30 07:49:03,858 epoch 1 - iter 6/63 - loss 3.62429563
2019-08-30 07:49:52,111 epoch 1 - iter 12/63 - loss 2.56970778
2019-08-30 07:50:14,790 epoch 1 - iter 18/63 - loss 2.23636854
2019-08-30 07:52:26,468 epoch 1 - iter 24/63 - loss 2.02165040
2019-08-30 07:52:54,131 epoch 1 - iter 30/63 - loss 1.85336732
2019-08-30 07:53:17,209 epoch 1 - iter 36/63 - loss 1.75582739
2019-08-30 07:53:38,659 epoch 1 - iter 42/63 - los

2019-08-30 08:45:25,182 epoch 2 - iter 36/63 - loss 1.32795631
2019-08-30 08:45:30,876 epoch 2 - iter 42/63 - loss 1.31881424
2019-08-30 08:45:57,924 epoch 2 - iter 48/63 - loss 1.30883798
2019-08-30 08:48:18,536 epoch 2 - iter 54/63 - loss 1.29632945
2019-08-30 08:48:28,273 epoch 2 - iter 60/63 - loss 1.29257091
2019-08-30 08:48:55,249 ----------------------------------------------------------------------------------------------------
2019-08-30 08:48:55,425 EPOCH 2 done: loss 1.2938 - lr 0.0010 - bad epochs 0
2019-08-30 08:50:17,722 DEV : loss 1.2849305868148804 - score 0.416
2019-08-30 08:53:50,785 TEST : loss 1.346233606338501 - score 0.4097
2019-08-30 08:54:02,082 ----------------------------------------------------------------------------------------------------
2019-08-30 08:54:04,375 epoch 3 - iter 0/63 - loss 1.16440558
2019-08-30 08:54:11,128 epoch 3 - iter 6/63 - loss 1.36838572
2019-08-30 08:54:17,907 epoch 3 - iter 12/63 - loss 1.32065621
2019-08-30 08:54:24,089 epoch 3 - 

2019-08-30 09:30:12,055 epoch 4 - iter 12/63 - loss 0.49906795
2019-08-30 09:30:17,569 epoch 4 - iter 18/63 - loss 0.50968976
2019-08-30 09:30:23,347 epoch 4 - iter 24/63 - loss 0.47567989
2019-08-30 09:30:29,608 epoch 4 - iter 30/63 - loss 0.49497867
2019-08-30 09:30:35,706 epoch 4 - iter 36/63 - loss 0.47769011
2019-08-30 09:30:40,469 epoch 4 - iter 42/63 - loss 0.48527218
2019-08-30 09:30:45,918 epoch 4 - iter 48/63 - loss 0.47183558
2019-08-30 09:30:51,205 epoch 4 - iter 54/63 - loss 0.47279594
2019-08-30 09:30:56,249 epoch 4 - iter 60/63 - loss 0.47369679
2019-08-30 09:30:58,388 ----------------------------------------------------------------------------------------------------
2019-08-30 09:30:58,389 EPOCH 4 done: loss 0.4740 - lr 0.0010 - bad epochs 0
2019-08-30 09:31:51,645 DEV : loss 0.5004297494888306 - score 0.8225
2019-08-30 09:34:03,928 TEST : loss 0.6718747615814209 - score 0.7799
2019-08-30 09:34:17,664 --------------------------------------------------------------------

2019-08-30 10:07:47,468 ----------------------------------------------------------------------------------------------------


## Run DISEQuA Benchmark - Cross-validation

Different classifier models are tested with different dependency levels of external linguistic resources (Low, Medium and High)

#### BERT + CNN - Cross validation

In [4]:
from typing import List
from flair_cnn_doc_embedding import DocumentCNNEmbeddings
from torch.utils.data import Dataset
import torch
from flair.data import Sentence, Corpus
from flair.embeddings import DocumentRNNEmbeddings, BertEmbeddings
from flair.models import TextClassifier
from flair.trainers import ModelTrainer
import time
import datetime
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, matthews_corrcoef, confusion_matrix


def build_flair_sentences(text_label_tuples):
    sentences = [Sentence(text, labels=[label], use_tokenizer=True) for text,label in text_label_tuples]
    return [s for s in sentences if len(s.tokens) > 0]

def get_labels(sentences:List[Sentence]):
    return [[l.value for l in s.labels] for s in sentences]

def calc_metrics_with_sklearn(clf:TextClassifier,sentences:List[Sentence],train_size=0,
                              fold=0,train_time=0,metric_average='macro',
                              classes=['DATE', 'ORGANIZATION', 'PERSON', 'MEASURE', 'OTHER', 'OBJECT', 'LOCATION']):
    targets = get_labels(sentences)
    start_time = time.time()
    clf.predict(sentences)
    test_time = time.time() - start_time
    prediction = get_labels(sentences)
    p = prediction
    t = targets
    data = {'datetime': datetime.datetime.now(),
            'model': 'cnn_bert',
            'accuracy': accuracy_score(prediction, targets),
            'precision': precision_score(prediction, targets, average=metric_average),
            'recall': recall_score(prediction, targets, average=metric_average),
            'f1': f1_score(prediction, targets, average=metric_average),
            'mcc': matthews_corrcoef(prediction, targets),
            'confusion': confusion_matrix(prediction, targets, labels=classes),
            'fold': fold,
            'train_size': size_train,
            'execution_time': train_time,
            'test_time': test_time}

    #report = metrics.classification_report(y_true=targets, y_pred=prediction, digits=3, output_dict=True)
    return data




for language in ['nl', 'es', 'en', 'it']:  
    print(f"########## {language} ##########")
    results = pd.DataFrame()
    dataset = load_disequa(language)
    save = 'results/DISEQuA_cnn_bert_'+language+'_100.csv'
    for size_train in [100]:  # 200, 300, 400
        print(f"##### {size_train} #####")
        
        word_embeddings = [BertEmbeddings('bert-base-multilingual-cased', layers='-1')]
        document_embeddings = DocumentCNNEmbeddings(word_embeddings, dropout=0.0, hidden_size=64)
        
        size_test = len(dataset) - size_train
        rs = StratifiedShuffleSplit(n_splits=10, train_size=size_train, test_size=size_test, random_state=1)
        fold = 0
        for train_indexs, test_indexs in rs.split(dataset, dataset['class']):
            fold += 1
            print(f"## {fold} ##")
            df_train = dataset.iloc[train_indexs]
            df_test = dataset.iloc[test_indexs]
            
            x_train:Dataset = build_flair_sentences([(text, label) for text, label in zip(df_train['question'], df_train['class'])])
            x_dev:Dataset = x_train
            x_test:Dataset = build_flair_sentences([(text, label) for text, label in zip(df_test['question'], df_test['class'])])
            
            corpus = Corpus(x_train, x_dev, x_test)
            label_dict = corpus.make_label_dictionary()

            clf = TextClassifier(document_embeddings, label_dictionary=label_dict, multi_label=False)
            trainer = ModelTrainer(clf, corpus,torch.optim.RMSprop)
            base_path = 'flair_resources/qc_'+language+'_disequa'
            start_time = time.time()
            trainer.train(base_path,
                          learning_rate=0.001,
                          mini_batch_size=32,
                          anneal_factor=0.5,
                          max_epochs=5,
                          patience=2,
                          )
            train_time = time.time() - start_time
            data = calc_metrics_with_sklearn(clf, x_test, train_size=size_train, train_time=train_time, fold=fold)
            results = results.append([data])
            results.to_csv(save)

########## nl ##########
##### 100 #####
2019-09-05 10:34:07,517 The pre-trained model you are loading is a cased model but you have not set `do_lower_case` to False. We are setting `do_lower_case=False` for you but you may want to check this behavior.
## 1 ##
2019-09-05 10:34:41,544 {'DATE', 'LOCATION', 'PERSON', 'OTHER', 'MEASURE', 'OBJECT', 'ORGANIZATION'}
2019-09-05 10:34:41,737 ----------------------------------------------------------------------------------------------------
2019-09-05 10:34:41,737 Evaluation method: MICRO_F1_SCORE
2019-09-05 10:34:42,358 ----------------------------------------------------------------------------------------------------
2019-09-05 10:34:43,670 epoch 1 - iter 0/4 - loss 1.96829784
2019-09-05 10:34:44,364 epoch 1 - iter 1/4 - loss 5.90502781
2019-09-05 10:34:45,092 epoch 1 - iter 2/4 - loss 8.72477178
2019-09-05 10:34:45,233 epoch 1 - iter 3/4 - loss 11.07618162
2019-09-05 10:34:45,339 -------------------------------------------------------------

2019-09-05 10:38:03,379 ----------------------------------------------------------------------------------------------------
## 2 ##
2019-09-05 10:38:11,224 {'DATE', 'LOCATION', 'PERSON', 'OTHER', 'MEASURE', 'OBJECT', 'ORGANIZATION'}
2019-09-05 10:38:11,237 ----------------------------------------------------------------------------------------------------
2019-09-05 10:38:11,238 Evaluation method: MICRO_F1_SCORE
2019-09-05 10:38:11,421 ----------------------------------------------------------------------------------------------------
2019-09-05 10:38:12,538 epoch 1 - iter 0/4 - loss 5.25004721
2019-09-05 10:38:13,480 epoch 1 - iter 1/4 - loss 4.81284785
2019-09-05 10:38:14,180 epoch 1 - iter 2/4 - loss 6.36161216
2019-09-05 10:38:14,379 epoch 1 - iter 3/4 - loss 6.32822573
2019-09-05 10:38:14,520 ----------------------------------------------------------------------------------------------------
2019-09-05 10:38:14,521 EPOCH 1 done: loss 6.3282 - lr 0.0010 - bad epochs 0
2019-09-05 1

2019-09-05 10:41:31,162 ----------------------------------------------------------------------------------------------------
## 3 ##
2019-09-05 10:41:39,224 {'DATE', 'OTHER', 'LOCATION', 'PERSON', 'MEASURE', 'OBJECT', 'ORGANIZATION'}
2019-09-05 10:41:39,238 ----------------------------------------------------------------------------------------------------
2019-09-05 10:41:39,239 Evaluation method: MICRO_F1_SCORE
2019-09-05 10:41:39,419 ----------------------------------------------------------------------------------------------------
2019-09-05 10:41:40,577 epoch 1 - iter 0/4 - loss 6.60427380
2019-09-05 10:41:41,480 epoch 1 - iter 1/4 - loss 6.90915132
2019-09-05 10:41:42,156 epoch 1 - iter 2/4 - loss 7.88175217
2019-09-05 10:41:42,305 epoch 1 - iter 3/4 - loss 8.14183390
2019-09-05 10:41:42,445 ----------------------------------------------------------------------------------------------------
2019-09-05 10:41:42,446 EPOCH 1 done: loss 8.1418 - lr 0.0010 - bad epochs 0
2019-09-05 1

2019-09-05 10:44:59,285 ----------------------------------------------------------------------------------------------------
## 4 ##
2019-09-05 10:45:07,608 {'DATE', 'OTHER', 'PERSON', 'LOCATION', 'MEASURE', 'OBJECT', 'ORGANIZATION'}
2019-09-05 10:45:07,623 ----------------------------------------------------------------------------------------------------
2019-09-05 10:45:07,623 Evaluation method: MICRO_F1_SCORE
2019-09-05 10:45:07,807 ----------------------------------------------------------------------------------------------------
2019-09-05 10:45:09,017 epoch 1 - iter 0/4 - loss 6.27249336
2019-09-05 10:45:09,727 epoch 1 - iter 1/4 - loss 5.41912436
2019-09-05 10:45:10,601 epoch 1 - iter 2/4 - loss 6.20903285
2019-09-05 10:45:10,763 epoch 1 - iter 3/4 - loss 8.32102072
2019-09-05 10:45:10,917 ----------------------------------------------------------------------------------------------------
2019-09-05 10:45:10,918 EPOCH 1 done: loss 8.3210 - lr 0.0010 - bad epochs 0
2019-09-05 1

2019-09-05 10:48:23,401 ----------------------------------------------------------------------------------------------------
## 5 ##
2019-09-05 10:48:30,842 {'DATE', 'LOCATION', 'OTHER', 'PERSON', 'MEASURE', 'OBJECT', 'ORGANIZATION'}
2019-09-05 10:48:30,866 ----------------------------------------------------------------------------------------------------
2019-09-05 10:48:30,867 Evaluation method: MICRO_F1_SCORE
2019-09-05 10:48:31,053 ----------------------------------------------------------------------------------------------------
2019-09-05 10:48:32,293 epoch 1 - iter 0/4 - loss 8.35355759
2019-09-05 10:48:33,116 epoch 1 - iter 1/4 - loss 6.29538012
2019-09-05 10:48:34,025 epoch 1 - iter 2/4 - loss 6.94496918
2019-09-05 10:48:34,238 epoch 1 - iter 3/4 - loss 8.95273924
2019-09-05 10:48:34,381 ----------------------------------------------------------------------------------------------------
2019-09-05 10:48:34,382 EPOCH 1 done: loss 8.9527 - lr 0.0010 - bad epochs 0
2019-09-05 1

...

2019-09-05 12:37:30,385 ----------------------------------------------------------------------------------------------------
## 7 ##
2019-09-05 12:37:37,675 {'DATE', 'OTHER', 'PERSON', 'LOCATION', 'MEASURE', 'OBJECT', 'ORGANIZATION'}
2019-09-05 12:37:37,702 ----------------------------------------------------------------------------------------------------
2019-09-05 12:37:37,703 Evaluation method: MICRO_F1_SCORE
2019-09-05 12:37:37,977 ----------------------------------------------------------------------------------------------------
2019-09-05 12:37:39,417 epoch 1 - iter 0/4 - loss 7.86969042
2019-09-05 12:37:40,212 epoch 1 - iter 1/4 - loss 7.28595209
2019-09-05 12:37:40,991 epoch 1 - iter 2/4 - loss 6.15274429
2019-09-05 12:37:41,214 epoch 1 - iter 3/4 - loss 4.80513847
2019-09-05 12:37:41,416 ----------------------------------------------------------------------------------------------------
2019-09-05 12:37:41,418 EPOCH 1 done: loss 4.8051 - lr 0.0010 - bad epochs 0
2019-09-05 1

2019-09-05 12:41:11,228 ----------------------------------------------------------------------------------------------------
## 8 ##
2019-09-05 12:41:18,729 {'DATE', 'OTHER', 'PERSON', 'LOCATION', 'MEASURE', 'OBJECT', 'ORGANIZATION'}
2019-09-05 12:41:18,757 ----------------------------------------------------------------------------------------------------
2019-09-05 12:41:18,758 Evaluation method: MICRO_F1_SCORE
2019-09-05 12:41:19,029 ----------------------------------------------------------------------------------------------------
2019-09-05 12:41:20,283 epoch 1 - iter 0/4 - loss 7.99959564
2019-09-05 12:41:21,089 epoch 1 - iter 1/4 - loss 4.96008027
2019-09-05 12:41:22,038 epoch 1 - iter 2/4 - loss 4.37036856
2019-09-05 12:41:22,209 epoch 1 - iter 3/4 - loss 3.84032792
2019-09-05 12:41:22,407 ----------------------------------------------------------------------------------------------------
2019-09-05 12:41:22,408 EPOCH 1 done: loss 3.8403 - lr 0.0010 - bad epochs 0
2019-09-05 1

2019-09-05 12:44:36,785 ----------------------------------------------------------------------------------------------------
## 9 ##
2019-09-05 12:44:44,771 {'DATE', 'OTHER', 'LOCATION', 'PERSON', 'MEASURE', 'OBJECT', 'ORGANIZATION'}
2019-09-05 12:44:44,802 ----------------------------------------------------------------------------------------------------
2019-09-05 12:44:44,803 Evaluation method: MICRO_F1_SCORE
2019-09-05 12:44:45,086 ----------------------------------------------------------------------------------------------------
2019-09-05 12:44:46,315 epoch 1 - iter 0/4 - loss 8.40016365
2019-09-05 12:44:47,403 epoch 1 - iter 1/4 - loss 5.19099069
2019-09-05 12:44:48,222 epoch 1 - iter 2/4 - loss 4.11136742
2019-09-05 12:44:48,412 epoch 1 - iter 3/4 - loss 3.56828424
2019-09-05 12:44:48,604 ----------------------------------------------------------------------------------------------------
2019-09-05 12:44:48,605 EPOCH 1 done: loss 3.5683 - lr 0.0010 - bad epochs 0
2019-09-05 1

2019-09-05 12:48:41,352 ----------------------------------------------------------------------------------------------------
## 10 ##
2019-09-05 12:48:49,755 {'DATE', 'OTHER', 'LOCATION', 'PERSON', 'MEASURE', 'OBJECT', 'ORGANIZATION'}
2019-09-05 12:48:49,784 ----------------------------------------------------------------------------------------------------
2019-09-05 12:48:49,785 Evaluation method: MICRO_F1_SCORE
2019-09-05 12:48:50,060 ----------------------------------------------------------------------------------------------------
2019-09-05 12:48:51,445 epoch 1 - iter 0/4 - loss 1.92382252
2019-09-05 12:48:52,235 epoch 1 - iter 1/4 - loss 5.20575100
2019-09-05 12:48:53,050 epoch 1 - iter 2/4 - loss 4.12004673
2019-09-05 12:48:53,247 epoch 1 - iter 3/4 - loss 3.57699010
2019-09-05 12:48:53,434 ----------------------------------------------------------------------------------------------------
2019-09-05 12:48:53,436 EPOCH 1 done: loss 3.5770 - lr 0.0010 - bad epochs 0
2019-09-05 

2019-09-05 12:52:03,087 ----------------------------------------------------------------------------------------------------


#### SVM + <font color=#007700>TF-IDF</font>

In [152]:
for language in ['en', 'es', 'it', 'nl']:
    print('\n\nLanguage: ', language)
    dataset = load_disequa(language)
    create_feature('tfidf', dataset, dataset, max_features=2000)
    
    model = {'name': 'svm', 'model': svm_linear}
    
    tfidf = np.array([list(r) for r in dataset['tfidf'].values])
    tfidf = normalize(tfidf, norm='max')
    
    X = np.array([list(x) for x in dataset['tfidf'].values])
    y = dataset['class'].values
    
    run_benchmark_cv(model, X, y, sizes_train=[100,200,300,400],
                     save='results/DISEQuA_svm_tfidf_' + language + '.csv')



Language:  en

100|..........
200|..........
300|..........
400|..........
Run time benchmark: 1.027012586593628


Language:  es

100|..........
200|..........
300|..........
400|..........
Run time benchmark: 1.0114972591400146


Language:  it

100|..........
200|..........
300|..........
400|..........
Run time benchmark: 1.1434721946716309


Language:  nl

100|..........
200|..........
300|..........
400|..........
Run time benchmark: 1.1250619888305664


#### SVM + <font color=#007700>TF-IDF</font> + <font color=#0055CC>WB</font>

In [163]:
for language in ['en', 'es', 'it', 'nl']:
    print('\n\nLanguage: ', language)
    embedding = load_embedding(path_wordembedding + 'wiki.multi.' + language + '.vec')
    dataset = load_disequa(language)
    create_feature('tfidf', dataset, dataset, max_features=2000)
    create_feature('embedding_sum', None, dataset, embedding)
    
    model = {'name': 'svm', 'model': svm_linear}
    
    tfidf = np.array([list(r) for r in dataset['tfidf'].values])
    tfidf = normalize(tfidf, norm='max')
    
    embedding = np.array([list(r) for r in dataset['embedding_sum'].values])
    embedding = normalize(embedding, norm='max')
    
    X = np.array([list(x) + list(xx) for x, xx in zip(tfidf, embedding)])
    y = dataset['class'].values
    
    run_benchmark_cv(model, X, y, sizes_train=[100,200,300,400],
                     save='results/DISEQuA_svm_cortes_' + language + '.csv')



Language:  en

100|..........
200|..........
300|..........
400|..........
Run time benchmark: 6.358882427215576


Language:  es

100|..........
200|..........
300|..........
400|..........
Run time benchmark: 7.197380065917969


Language:  it

100|..........
200|..........
300|..........
400|..........
Run time benchmark: 5.5334153175354


Language:  nl

100|..........
200|..........
300|..........
400|..........
Run time benchmark: 6.624628782272339


#### SVM + <font color=#007700>TF-IDF</font> + <font color=#0055CC>WB</font> + <font color=#CC6600>POS</font> + <font color=#CC6600>NER</font>

In [164]:


for language in ['en', 'es', 'it', 'nl']:
    print('\n\nLanguage: ', language)
    embedding = load_embedding(path_wordembedding + 'wiki.multi.' + language + '.vec')
    dataset = load_disequa(language)
    create_feature('tfidf', dataset, dataset, max_features=2000)
    create_feature('embedding_sum', dataset, dataset, embedding)
    create_feature('pos_hotencode', dataset, dataset)
    create_feature('ner_hotencode', dataset, dataset)
    model = {'name': 'svm', 'model': svm_linear}
    
    tfidf = np.array([list(r) for r in dataset['tfidf'].values])
    tfidf = normalize(tfidf, norm='max')
    
    embedding = np.array([list(r) for r in dataset['embedding_sum'].values])
    embedding = normalize(embedding, norm='max')
    
    pos = np.array([list(r) for r in dataset['pos_hotencode'].values])
    
    ner = np.array([list(r) for r in dataset['ner_hotencode'].values])
    
    X = np.array([list(x) + list(xx) + list(xxx) + list(xxxx) for x, xx, xxx, xxxx in zip(tfidf, embedding, pos, ner)])
    
    y = dataset['class'].values
    
    run_benchmark_cv(model, X, y, sizes_train=[100,200,300,400],
                     save='results/DISEQuA_svm_high_' + language + '.csv')



Language:  en

100|..........
200|..........
300|..........
400|..........
Run time benchmark: 6.811999559402466


Language:  es

100|..........
200|..........
300|..........
400|..........
Run time benchmark: 8.384974479675293


Language:  it

100|..........
200|..........
300|..........
400|..........
Run time benchmark: 6.426969528198242


Language:  nl

100|..........
200|..........
300|..........
400|..........
Run time benchmark: 6.852076053619385


## Old stuffs bellow

#### CNN

In [ ]:
# 'en', 'es'
for language in ['es']:
    print('\n\nLanguage: ', language)
    #embedding = load_embedding(path_wordembedding + 'wiki.multi.' + language + '.vec')
    dataset_train, dataset_test = load_uiuc(language)
    text_representation = 'vocab_index'
    vocabulary_inv = create_feature(text_representation, dataset_train, dataset_train)
    create_feature(text_representation, dataset_train, dataset_test)
    model = {'name': 'cnn', 'model': cnn}
    X_train = np.array([list(x) for x in dataset_train[text_representation].values])
    X_test = np.array([list(x) for x in dataset_test[text_representation].values])
    #X_train = pad_sequences(X_train, maxlen=12, dtype='float', padding='post', truncating='post', value=0.0)
    #X_test = pad_sequences(X_test, maxlen=12, dtype='float', padding='post', truncating='post', value=0.0)
    y_train = dataset_train['class'].values
    y_test = dataset_test['class'].values
    ohe = OneHotEncoder()
    y_train = ohe.fit_transform([[y_] for y_ in y_train]).toarray()
    y_test = ohe.transform([[y_] for y_ in y_test]).toarray()
    # , 
    run_benchmark(model, X_train, y_train, X_test, y_test, sizes_train=[1000, 2000, 3000, 4000, 5500],
                  runs=30, save='results/UIUC_cnn_' + language + '.csv', epochs=100, onehot=ohe,
                  vocabulary_size=len(vocabulary_inv))

#### LSTM + WordEmbedding

In [7]:
for language in ['en', 'pt', 'es']:
    print('\n\nLanguage: ', language)
    embedding = load_embedding(path_wordembedding + 'wiki.multi.' + language + '.vec')
    dataset_train, dataset_test = load_uiuc(language)
    # dataset_train = dataset_train[:100]
    #dataset_test = dataset_test[:10]
    create_feature('embedding', dataset_train, dataset_train, embedding)
    create_feature('embedding', dataset_train, dataset_test, embedding)
    model = {'name': 'lstm', 'model': lstm_default}
    #print(dataset_train['embedding'].values.shape)
    #print(dataset_train['embedding'].values.dtype)
    #print(dataset_test['embedding'].values.shape)
    X_train = np.array([list(x) for x in dataset_train['embedding'].values])
    X_test = np.array([list(x) for x in dataset_test['embedding'].values])
    X_train = pad_sequences(X_train, maxlen=12, dtype='float', padding='post', truncating='post', value=0.0)
    X_test = pad_sequences(X_test, maxlen=12, dtype='float', padding='post', truncating='post', value=0.0)
    y_train = dataset_train['class'].values
    y_test = dataset_test['class'].values
#     y_train_sub = dataset_train['sub_class'].values
#     sub_classes = set()
#     for sc in y_train_sub:
#         sub_classes.add(sc)
#     y_test_sub = dataset_test['sub_class'].values
#     X_test_sub_ = []
#     y_test_sub_ = []
#     for i in range(len(X_test)):
#         if y_train_sub[i] in sub_classes:
#             X_test_sub_.append(X_test[i])
#             y_test_sub_.append(y_train_sub[i])
#     X_test_sub_ = np.array(X_test_sub_)
#     y_test_sub_ = np.array(y_test_sub_)
    ohe = OneHotEncoder()
    y_train = ohe.fit_transform([[y_] for y_ in y_train]).toarray()
    y_test = ohe.transform([[y_] for y_ in y_test]).toarray() 
    run_benchmark(model, X_train, y_train, X_test, y_test, sizes_train=[1000, 2000, 3000, 4000, 5500],
                  runs=10, save='results/UIUC_lstm_embedding_' + language + '_10.csv', epochs=100, onehot=ohe)
    #run_benchmark(model, X_train, y_train_sub, X_test_sub_, y_test_sub_, sizes_train=[1000, 2000, 3000, 4000, 5500],
    #              save='results/UIUCsub_svm_tfidf_' + language + '.csv')



Language:  en

1000|..........
2000|..........
3000|..........
4000|..........
5500|..........Run time benchmark: 14027.941225767136


Language:  pt

1000|..........
2000|..........
3000|..........
4000|..........
5500|..........Run time benchmark: 15653.799264907837


Language:  es

1000|..........
2000|..........
3000|..........
4000|..........
5500|..........Run time benchmark: 17422.95069336891


#### LSTM + BERT

In [ ]:
for language in ['en']:
    print('\n\nLanguage: ', language)
    #embedding = load_embedding(path_wordembedding + 'wiki.multi.' + language + '.vec')
    dataset_train, dataset_test = load_uiuc(language)
    # debug
    print('WARNING: use subset (first 1000 entries) of training data')
    #dataset_train = dataset_train[:5500].copy()
    
    create_feature('bert', dataset_train, dataset_train)
    create_feature('bert', dataset_train, dataset_test)
    model = {'name': 'lstm', 'model': lstm_default}
    X_train = dataset_train['bert'].values
    X_test = dataset_test['bert'].values
    
    X_train = np.array([x for x in X_train])
    X_test = np.array([x for x in X_test])
    
    #X_train = pad_sequences(X_train, maxlen=12, dtype='float', padding='post', truncating='post', value=0.0)
    #X_test = pad_sequences(X_test, maxlen=12, dtype='float', padding='post', truncating='post', value=0.0)
    y_train = dataset_train['class'].values
    y_test = dataset_test['class'].values
    ohe = OneHotEncoder()
    y_train = ohe.fit_transform([[y_] for y_ in y_train]).toarray()
    y_test = ohe.transform([[y_] for y_ in y_test]).toarray() 
    run_benchmark(model, X_train, y_train, X_test, y_test, sizes_train=[5500], # 1000, 2000, 3000, 4000, 5500
                  runs=1, save='results/UIUC_lstm_bert_' + language + '.csv', 
                  epochs=100, onehot=ohe, in_dim=768)
    #run_benchmark(model, X_train, y_train_sub, X_test_sub_, y_test_sub_, sizes_train=[1000, 2000, 3000, 4000, 5500],
    #              save='results/UIUCsub_svm_tfidf_' + language + '.csv')

## DISEQuA Benchmark

### RUN DISEQuA Benchmark

##### SVM + TFIDF

In [ ]:
for language in ['DUT', 'ENG', 'ITA', 'SPA']:
    print('\n\nLanguage: ', language)
    dataset = load_disequa(language)
    create_feature('tfidf', dataset, dataset, embedding)
    model = {'name': 'svm', 'model': svm_linear}
    X = np.array([list(x) for x in dataset['tfidf'].values])
    y = dataset['class'].values
    run_benchmark(model, X, y, sizes_train=[100,200,300,400,405],
                  save='results/DISEQuA_svm_tfidf_' + language + '.csv')

##### RFC + TFIDF

In [ ]:
for language in ['DUT', 'ENG', 'ITA', 'SPA']:
    print('\n\nLanguage: ', language)
    dataset = load_disequa(language)
    create_feature('tfidf', dataset, dataset, embedding)
    model = {'name': 'rfc', 'model': random_forest}
    X = np.array([list(x) for x in dataset['tfidf'].values])
    y = dataset['class'].values
    run_benchmark(model, X, y, sizes_train=[100,200,300,400],
                  save='results/DISEQuA_rfc_tfidf_' + language + '.csv')

##### SVM + TFIDF_3gram + SKB

In [ ]:
for language in ['DUT', 'ENG', 'ITA', 'SPA']:
    print('\n\nLanguage: ', language)
    dataset = load_disequa(language)
    create_feature('tfidf_3gram', dataset, dataset)
    model = {'name': 'svm', 'model': svm_linear}
    X = np.array([list(x) for x in dataset['tfidf'].values])
    y = dataset['class'].values
    skb = SelectKBest(chi2, k=2000).fit(X, y)
    X = skb.transform(X)
    run_benchmark(model, X, y, sizes_train=[100,200,300,400],
                  save='results/DISEQuA_svm_tfidf_3gram_' + language + '.csv')

##### LSTM + Embedding

In [ ]:
for language, embd_l in zip(['SPA'], ['es']):
    print('\n\nLanguage: ', language)
    embedding = load_embedding(path_wordembedding + 'wiki.multi.' + embd_l + '.vec')
    dataset = load_disequa(language)
    create_feature('embedding', dataset, dataset, embedding)
    model = {'name': 'lstm', 'model': lstm_default}
    X = np.array([list(x) for x in dataset['embedding'].values])
    y = dataset['class'].values
    X = pad_sequences(X, maxlen=12, dtype='float', padding='post', truncating='post', value=0.0)
    ohe = OneHotEncoder()
    y = ohe.fit_transform([[y_] for y_ in y]).toarray()
    run_benchmark(model, X, y, sizes_train=[100,200,300,400,405], onehot=ohe,
                  save='results/DISEQuA_lstm_embedding_' + language + '.csv')

##### CNN

In [ ]:
for language, embd_l in zip(['DUT', 'ENG', 'ITA', 'SPA'], ['nl', 'eng', 'it', 'es']):
    print('\n\nLanguage: ', language)
    #embedding = load_embedding(path_wordembedding + 'wiki.multi.' + embd_l + '.vec')
    dataset = load_disequa(language)
    text_representation = 'vocab_index'
    vocabulary_inv = create_feature(text_representation, dataset, dataset)
    model = {'name': 'cnn', 'model': cnn}
    X = np.array([list(x) for x in dataset[text_representation].values])
    y = dataset['class'].values
    #X = pad_sequences(X, maxlen=12, dtype='float', padding='post', truncating='post', value=0.0)
    ohe = OneHotEncoder()
    y = ohe.fit_transform([[y_] for y_ in y]).toarray()
    run_benchmark(model, X, y, sizes_train=[100,200,300,400], onehot=ohe, vocabulary_size=len(vocabulary_inv),
                  save='results/DISEQuA_cnn_' + language + '.csv', epochs=100)